In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
# requests for human-friendly HTTP requests
import requests
from requests.exceptions import RequestException
from contextlib import closing
# BeautifulSoup for HTML parsing
from bs4 import BeautifulSoup, NavigableString
# mimetypes for detecting proper extension
import mimetypes
# Image to show a downloaded image
from IPython.display import Image

In [ ]:
# from https://realpython.com/python-web-scraping-practical-introduction/
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [ ]:
# Get the page and push it into BeautifulSoup
url = "https://de.wikipedia.org/wiki/Liste_der_Mitglieder_des_Deutschen_Bundestages_(19._Wahlperiode)"
content = simple_get(url)
html = BeautifulSoup(content, 'html.parser')

In [ ]:
# find the right table
mdbtable = html.select('table')[2]
mdbtable

In [ ]:
# get a sample row (first one's probably a header)
row = mdbtable.select('tr')[1]
row

In [ ]:
# figure out how to extract image URL from row
image_url = row.select('td')[0].select('a')[0].contents[0]["src"]
image_url

In [ ]:
# figure out how to extract label from row
label = row.select('td')[3].contents[0].strip()
label

In [ ]:
# extract image and label from all rows
img_and_party = []
errors = []
for row in mdbtable.select('tr')[1:]:
    try:
        img = row.select('td')[0].select('a')[0].contents[0]["src"]
        party = row.select('td')[3].contents[0]
        
        # special case, some parties are links, others are text
        if type(party)==NavigableString:
            party = party.strip()
        else:
            party = party.contents[0]
    except Exception as e:
        errors.append((row, e))
        continue
        
    # handle special cases (here: Placeholder image)
    if 'Placeholder' in img:
        continue
        
    # if all ok, add to list
    img_and_party.append((img, party))
    
# let's see if we accumulated any errors
len(errors)   


In [ ]:
# let's see a sample entry
img_and_party[0]

In [ ]:
# now download images and store according to label
# NOTE: this does not create a validation set, so you'll want to 
# pass on valid_pct to ImageDataBunch.from_folder, 
# such that a validation set of valid_pct percent is created automatically
urlpattern = "https:{}"
pathpattern = "/home/nneubaue/downloads/party/{}/{}{}"
dirpattern = "/home/nneubaue/downloads/party/{}"

for (i, (img, party)) in enumerate(img_and_party):    
    url = urlpattern.format(img)
    
    directory = dirpattern.format(party)
    if not os.path.exists(directory):
        os.makedirs(directory)        
    
    response = requests.get(url)
    content_type = response.headers['content-type']
    extension = mimetypes.guess_extension(content_type)
    if extension == '.jpe':
        extension = '.jpg'
        
    path = pathpattern.format(party, i, extension)
    if response.status_code == 200:
        with open(path, 'wb') as f:
            f.write(response.content)            

In [ ]:
# let's see where the last image was written to...
path

In [ ]:
# ... and what it looks like
Image(path)